In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import os
import tensorflow as tf

# Pretrained Models :  https://www.kaggle.com/ekaterinadranitsyna/keras-applications-models

In [ ]:
df=pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')

In [ ]:
df

# Data Preprocessing

## Check for Null Values

### No Null Values

In [ ]:
df.isna().sum().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(18, 18))
sns.heatmap(df[1:].corr(),annot=True)

# Data Visualization and EDA

In [ ]:
import seaborn as sns
sns.barplot(df['Subject Focus'],df['Pawpularity'])

In [ ]:
sns.histplot(df['Pawpularity'])

## Most Animals Pawpularity score is in between 20 to 50

In [ ]:
sns.histplot(df['Subject Focus'])

In [ ]:
np.unique(df['Eyes'])

In [ ]:
sns.barplot(df['Human'],df['Pawpularity'])

In [ ]:
sns.jointplot(df['Human'],df['Pawpularity'])

In [ ]:
df2=df['Pawpularity']
df1=df.drop('Pawpularity',axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x,test_x,train_y,test_y=train_test_split(df1,df2,test_size=0.3)

In [ ]:
#ddf=test_x['Id']

In [ ]:
#train_x=train_x.drop('Id',axis=1)
#test_x=test_x.drop('Id',axis=1)

In [ ]:
print(train_x.shape[0],test_x.shape[0])

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
train_x.iloc[:,1:]

In [ ]:
ddff=pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')

In [ ]:
ddff

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
test_x.iloc[:,1:]

In [ ]:
from sklearn.metrics import mean_squared_error
lin=LinearRegression()
reg=PolynomialFeatures(degree = 1)
dd=reg.fit_transform(train_x.iloc[:,1:])
lin.fit(dd, train_y)
dd1=reg.fit_transform(test_x.iloc[:,1:])
df_y=lin.predict(dd1)
ddf1=reg.fit_transform(ddff.iloc[:,1:])
ddff['Pawpularity']=lin.predict(ddf1)
print(np.sqrt(mean_squared_error(df_y, test_y)))

In [ ]:
ddff[['Id','Pawpularity']].to_csv('submission.csv',index=False)

In [ ]:
from IPython.display import FileLink
FileLink(r'submission.csv')

In [ ]:
dd.shape

In [ ]:
print(df_y.shape)
print(test_x.iloc[:,1:].shape)

In [ ]:
test_y

In [ ]:
train_folder='../input/petfinder-pawpularity-score/train'
test_folder='../input/petfinder-pawpularity-score/test'

In [ ]:
d_tr=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
d_ts=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
autotune = tf.data.experimental.AUTOTUNE
img_size=299
batch_size=8

In [ ]:
y = 'Pawpularity'
def id_to_path(img_id,dir):
    return os.path.join(dir, f'{img_id}.jpg')

def get_image(path):
    image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
    image = tf.cast(tf.image.resize_with_pad(image, img_size, img_size), dtype=tf.float32)
    return tf.keras.applications.inception_v3.preprocess_input(image)

def process_dataset(path, label):
    return get_image(path), label

def get_dataset(x, y=None):
    if y is not None:
        ds = tf.data.Dataset.from_tensor_slices((x, y))
        return ds.map(process_dataset, num_parallel_calls=autotune) \
            .batch(batch_size).prefetch(buffer_size=autotune)
    else:
        ds = tf.data.Dataset.from_tensor_slices(x)
        return ds.map(get_image, num_parallel_calls=autotune) \
            .batch(batch_size).prefetch(buffer_size=autotune)

d_tr['path'] = d_tr['Id'].apply(lambda x: id_to_path(x, train_folder))
d_ts['path'] = d_ts['Id'].apply(lambda x: id_to_path(x, test_folder))

train_subset, valid_subset = train_test_split(
    d_tr[['path', y]],
    test_size=0.3, shuffle=True, random_state=5
)    

In [ ]:
train_data = get_dataset(x=train_subset['path'], y=train_subset[y]/100.).shuffle(1000)
valid_data = get_dataset(x=valid_subset['path'], y=valid_subset[y]/100.).shuffle(1000)
test_data = get_dataset(x=d_ts['path'])

In [ ]:
EfficientNet_path='../input/keras-applications-models/Xception.h5'

In [ ]:
# Load pre-trained model: EfficientNet
# This needs an input shape of (224,224,3)
model = tf.keras.models.load_model(EfficientNet_path)
model.trainable=False

In [ ]:
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.layers import BatchNormalization,Dropout
flat1 = Flatten()(model.layers[-1].output)
x = Dense(512,activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2())(flat1)
x = Dense(512,activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2())(x)
output = Dense(1,activation='elu')(x)
# define new model
model = Model(inputs=model.inputs, outputs=output)
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true*100 - y_pred*100)))
opt = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=opt,
              loss=rmse,
              metrics=[rmse])
hist = model.fit(train_data,validation_data=valid_data, epochs=5)

In [ ]:
model.evaluate(valid_data)

In [ ]:
d_ts[y]=model.predict(test_data)*100

In [ ]:
d_ts[['Id', y]].to_csv('submission.csv', index=False)
d_ts[['Id', y]].head()